In [24]:
import numpy as np
# import units as u
import numpy as np
import pandas as pd

import rasterio
from rasterio.plot import show_hist, show
import geopandas as gpd
import rasterstats
from shapely.geometry import shape

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

from icecream import ic

In [2]:
# shape file definining administrative borders 
states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
states = gpd.read_file(states_path)

In [25]:
states_m = states.to_crs('epsg:32632')
state_areas = pd.DataFrame(states_m.apply(lambda x: (x.admin1Name, shape(x["geometry"]).area ), axis=1).to_list(), columns=["state", "area (m2)"])
state_areas.head()

# TODO => save this in csv or pickle file..

# state_areas["area (m2)"].sum()/ 1e6 # ~ 923,768 km2

,state,area (m2)
0,Abia,4.858882e+09
1,Adamawa,3.792499e+10
2,Akwa Ibom,6.736774e+09
3,Anambra,4.807933e+09
4,Bauchi,4.849640e+10


In [6]:
# raster data sources => wind, temperature, solar flux 

data_paths = {
    "wind_speed": "data/Nigeria_MeanWindSpeed/NGA_wind-speed_100m.tif",
    "temperature": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/TEMP.tif",
    "solar_flux": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/DNI.tif",
}

In [8]:
def get_state_means(data_path, states=states):
    # read raster files..
    data = rasterio.open(data_path)
    data_array = data.read()[0]
    affine = data.transform

    # calculate zonal statistics
    averaged_data = rasterstats.zonal_stats(states, data_array, affine=affine, stats = ["mean"], geojson_out=True)

    state_wind_average = []
    for item in averaged_data:
        state_wind_average.append((item["properties"]["admin1Name"],
        item["properties"]["mean"]))

    df = pd.DataFrame(state_wind_average, columns=["state", "value"])
    return df


In [11]:
data_averages = {}
for name, path in data_paths.items():
    data_averages[name] = get_state_means(path, states=states)

# TODO => save this in csv or pickle file..

In [19]:
avg_vals = {n: i["value"] for n, i in data_averages.items()}

In [13]:
def calculate_power_panel(F_cur, T_a, w):
    """
    F_cur = _ # W/m^2 current solar flux normal to the panel 
    T_a = _ # K / ºC, air temperature the panel is exposed to 
    w = _ # m/s, wind speed panel is exposed to 
    => all of these are arrays 

    return power potential of a single panel in the area defined by the characteristics above
    
    """

    # values that change based on design! 
    D_f = 0.864 # derating factor, product of correction factors for additional processes affecting solar output, Table 5.2 
    E_panel = 0.18 # solar panel efficiency obtained under standard test conditions -- Ex 5.2
    A_panel = 1.5 # m^2, surface area of the pane -- Ex 5.2
    
    # 5.9, cell temperature, empirical so units do not eqate 
    T_c = T_a + 0.32 * (F_cur/(8.91 + 2*w))

    b_ref = 0.0025 # / K, temperature coefficient 
    T_th = 55 # K, threshold temeprature 
    T_ref = 298.15 # K, reference temperature 
    # 5.8, correction for cell temperature 
    C_temp = 1 - b_ref * np.maximum( np.minimum(T_c - T_ref, T_th ), 0 )

    # 5.7 actual AC power output from a solar panel at a given time 
    # P_ac = p_mpp_stc * C_temp * D_f / F_1000 # W based on panel rating
    P_ac = F_cur * A_panel * E_panel * C_temp * D_f # W, based on real conditions 

    return P_ac


In [28]:
power_potential_panel  = calculate_power_panel(F_cur=avg_vals["solar_flux"], T_a=avg_vals["temperature"], w=avg_vals["wind_speed"])

In [ ]:
# determine the areas of each state, and determine the power potential if covered with identical arrays of solar panels..

In [36]:
def calc_power_potential(A_spacing, single_panel_potential):
    A_panel = 1.5 # m2
    n_panels = A_spacing / A_panel 
    farm_potential = np.multiply(single_panel_potential,n_panels)
    return farm_potential


In [37]:
power_potential_panel_farm = calc_power_potential(A_spacing=state_areas["area (m2)"], single_panel_potential=power_potential_panel) # W 

In [41]:
pd.set_option('display.float_format', lambda x: f'{x:,.0f}')

In [72]:
def print_nice(float):
    return print(f"{float:,.0f}")

In [44]:
w_to_mw = 1/1e6
hours_per_year = 8760.0
mw_to_mwh = hours_per_year
power_potential_panel_farm_mwh = power_potential_panel_farm*w_to_mw*mw_to_mwh  # MW 
p3f = power_potential_panel_farm_mwh
p3f

0     14,664,504
1    216,181,365
2     18,836,287
3     15,385,295
4    280,529,102
5     26,520,016
6    122,937,080
7    438,402,680
8     67,789,466
9     50,670,104
10    21,464,071
11    61,822,971
12    20,599,605
13    25,357,629
14    30,046,811
15    99,729,362
16    15,369,809
17   136,380,803
18   228,872,721
19   117,766,965
20   141,914,461
21   194,384,528
22   108,112,180
23   137,475,311
24    12,028,823
25   112,000,397
26   320,905,478
27    50,386,845
28    47,323,629
29    27,141,943
30    95,342,015
31   138,954,359
32    29,053,395
33   185,352,742
34   282,570,754
35   270,486,722
36   186,765,332
dtype: float64

In [50]:
p3f_sort = p3f.sort_values(ascending=False)

In [55]:
p3f_sort[0:5].apply(lambda x: x*0.01).sum() # mwh energy produced 

15928947.365634173

In [62]:
state_areas.iloc[p3f_sort[0:5].index]["area (m2)"].apply(lambda x: x*0.01).sum()


2965654220.4539

In [63]:
def calc_num_panels(p_land, num_states, state_energy, state_areas):
    # calc energy available if use x percent land of top y most producing states 
    state_energy_sort = state_energy.sort_values(ascending=False)
    state_energy_lim = state_energy_sort[0:num_states]

    total_energy = state_energy_lim.apply(lambda x: x*p_land).sum()

    # calc amount of land used to produce this energy 
    land_used = state_areas.iloc[state_energy_lim.index]["area (m2)"].apply(lambda x: x*p_land).sum()

    # calc number of solar panels on this land, assuming standard panels 
    A_panel = 1.5 # m2
    n_panels = land_used/A_panel 

    return {
        "total_energy (mwh)": total_energy,
        "land_used (m2)": land_used,
        "n_panels": n_panels
    }


    

    

In [73]:
test_1 = calc_num_panels(p_land=0.02, num_states=5, state_energy=p3f, state_areas=state_areas)

In [74]:
{k: print_nice(v) for k,v in test_1.items() }

31,857,895
5,931,308,441
3,954,205,627


{'total_energy (mwh)': None, 'land_used (m2)': None, 'n_panels': None}

In [47]:
print_nice(p3f[26]*0.01)


3,209,055
